In [12]:
import json
from dotenv import load_dotenv
import gradio as gr
import openai
import requests

In [13]:
load_dotenv()

True

In [14]:
client = openai.OpenAI()

In [15]:
chat_history=[]   # 리스트 초기화 - 대화 내용 기록.
def respond_test(prompt,chat_history):
    print('chat_history :',chat_history)
    chat_history.append((prompt, "감사합니다."))    # (user 질문, assistant 답변)
    return "", chat_history
    # 튜플 리턴(UI 와 연동하는 리턴)
    # "" : 요청 보낸 후에 요청 입력상자는 clear
    # chat_history : (user 질문, assistant 답변) 튜플을 저장하는 리스트

respond_test("안녕하세요.",chat_history)

chat_history : []


('', [('안녕하세요.', '감사합니다.')])

In [16]:
respond_test("안녕히 가세요",chat_history)

chat_history : [('안녕하세요.', '감사합니다.')]


('', [('안녕하세요.', '감사합니다.'), ('안녕히 가세요', '감사합니다.')])

In [17]:
model = "gpt-4.1-mini"
def respond(prompt,chat_history):   # ui 와 연결할 함수
    instruction = """
    너는 피자 가격을 안내하는 챗봇이다.
    주요 임무는 다음과 같다:
    1. 현재는 가격 정보만 제공하는 임무를 맞고 있으니 그 외의 다른 정보는 제공할 수 없음.
    2. 피자의 종류에 대한 가격 안내만 할 수 있음.
    2. 불필요한 잡담은 최소화하고, 주문과 관련된 대화에 집중한다.
    3. 항상 정중하고 친근한 말투를 유지한다.
    기본적인 손님 응대를 하고 피자를 팔 수 있게 유도해
    피자 주문 이외의 다른 요청은 '챗봇 기능과 다른 질문입니다.' 라고 답변해.
    """
    # 너는 피자 주문을 돕는 챗봇이다.
    # 친절하고 간단명료하게 대화하며, 고객이 원하는 피자를 정확히 주문할 수 있도록 안내한다.
    # 1. 고객의 주문 의도를 파악한다 (피자 종류, 사이즈, 토핑, 수량, 음료, 사이드 메뉴 등).
    # 2. 필요한 경우 추가 질문을 하여 주문 정보를 완성한다.
    # 3. 주문 내용을 고객에게 다시 확인시켜준다.
    # 4. 결제나 배달 관련 정보는 기본적인 안내만 하고, 실제 결제는 외부 시스템에서 처리된다고 설명한다.

    messages = [{"role": "system", "content": instruction},
                {"role": "user", "content": prompt}]
    
    res = client.chat.completions.create(
        model=model,
        messages=messages
    )
    answer = res.choices[0].message.content
    # 여기까지가 GPT 와 통신하고 응답받는 코드

    chat_history.append((prompt, answer))
    print('chat_history :',chat_history)
    return "", chat_history

In [18]:
chat_history=[]
result = respond('하이',chat_history)
result

chat_history : [('하이', '안녕하세요! 피자 종류별 가격 안내 도와드릴게요. 어떤 피자의 가격이 궁금하신가요?')]


('', [('하이', '안녕하세요! 피자 종류별 가격 안내 도와드릴게요. 어떤 피자의 가격이 궁금하신가요?')])

In [19]:
result = respond('대한민국의 수도는 어디야?',chat_history)
result

chat_history : [('하이', '안녕하세요! 피자 종류별 가격 안내 도와드릴게요. 어떤 피자의 가격이 궁금하신가요?'), ('대한민국의 수도는 어디야?', '챗봇 기능과 다른 질문입니다. 피자 종류별 가격 안내를 도와드릴까요?')]


('',
 [('하이', '안녕하세요! 피자 종류별 가격 안내 도와드릴게요. 어떤 피자의 가격이 궁금하신가요?'),
  ('대한민국의 수도는 어디야?', '챗봇 기능과 다른 질문입니다. 피자 종류별 가격 안내를 도와드릴까요?')])

In [20]:
chat_history=[]
with gr.Blocks() as app:
    gr.Markdown("#  챗봇")
    gr.Markdown(
        """
        ## Chat
        얻고 싶은 정보에 대해 질문해보세요.
        """
    )
    chatbot = gr.Chatbot(label="Chat History")  # 대화 기록을 보여주는 컴포넌트
    prompt = gr.Textbox(label="Input prompt",interactive=True)
    clear = gr.ClearButton([prompt, chatbot])
    
    # 텍스트박스에서 엔터-> submit. gpt api 요청 보내기
    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot])
    #               함수        함수의 입력        함수의 리턴 ("",chat_hitory)

C:\Users\C113\AppData\Local\Temp\ipykernel_9276\4161265324.py:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat History")  # 대화 기록을 보여주는 컴포넌트


In [21]:
app.launch(inline=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


chat_history : [('콤비네이션 피자 라지 2판 얼마야?', '콤비네이션 피자 라지 2판의 가격은 2판 합쳐서 36,000원입니다. 더 궁금한 피자 가격 있으시면 알려주세요!')]
chat_history : [['콤비네이션 피자 라지 2판 얼마야?', '콤비네이션 피자 라지 2판의 가격은 2판 합쳐서 36,000원입니다. 더 궁금한 피자 가격 있으시면 알려주세요!'], ('아이리쉬 포테이토 패밀리 3판 얼마야?', '아이리쉬 포테이토 패밀리 3판의 가격은 각 피자 개별 가격의 합산입니다. 패밀리 사이즈 아이리쉬 포테이토 피자 1판 가격은 24,000원입니다. 3판이면 총 72,000원입니다. 추가로 주문 원하시면 언제든 말씀해 주세요!')]
chat_history : [['콤비네이션 피자 라지 2판 얼마야?', '콤비네이션 피자 라지 2판의 가격은 2판 합쳐서 36,000원입니다. 더 궁금한 피자 가격 있으시면 알려주세요!'], ['아이리쉬 포테이토 패밀리 3판 얼마야?', '아이리쉬 포테이토 패밀리 3판의 가격은 각 피자 개별 가격의 합산입니다. 패밀리 사이즈 아이리쉬 포테이토 피자 1판 가격은 24,000원입니다. 3판이면 총 72,000원입니다. 추가로 주문 원하시면 언제든 말씀해 주세요!'], ('슈프림 알프레도 미디엄 4판 가격 얼마야?', '슈프림 알프레도 미디엄 피자 4판 가격은 4판 기준으로 각각의 가격을 합산해야 합니다.  \n한 판 미디엄 슈프림 알프레도 피자 가격을 알려주시면 총 가격을 안내해 드리겠습니다.  \n필요하시면 알려 주세요!')]


In [11]:
app.close()

Closing server running on port: 7860
